# Custom Exceptions: Tailoring Error Signals
- Built-in exceptions are great, but often too generic for application-specific failures.  
- A custom exception like `ServiceConnectionError` immediately conveys context compared to a plain `Exception`.  
- Defining a base exception class groups related errors; subclasses add specificity for targeted handling.  
- Catching `except BaseError:` handles all related issues, while `except SpecificError:` addresses one case precisely.

## Simple Custom Exceptions (Inheritance)
- Create a new exception by subclassing `Exception` or another exception class.  
- Using `pass` is enough when no extra logic or attributes are needed.  
- Catch the base class (`AutomationError`) to handle any related subclass errors in one block.  
- Use subclasses (`FileProcessingError`, `APICallError`) when context-specific handling is required.  

In [3]:
class AutomationError(Exception):
    """Base for all automation script errors."""
    pass


class FileProcessingError(AutomationError):
    """Error during file processing stage."""
    pass


class APICallError(AutomationError):
    """Error during an external API call."""
    pass


def process_file(filepath):
    raise FileProcessingError(f"Failed to process file at path: {filepath}")


try:
    process_file("nonexistent.csv")
except FileProcessingError as e:
    print(f"File error: {e}")
except AutomationError:
    print("Other automation error occurred.")


File error: Failed to process file at path: nonexistent.csv


## Adding Context with `__init__`
- Override `__init__` in your exception class to capture **context** (e.g., filename, invalid value).  
- Store custom attributes on `self` and build a clear message passed to `super().__init__()`.  
- Inherit from a **built-in exception** (`ValueError`) when semantics align, allowing broad catches.  
- Attribute access (`e.key_name`) provides extra debugging info in handlers.  

In [3]:
class ConfigValueError(ValueError):
    """Raised when a config value is invalid."""

    # Parameters key_name, invalid_value are instance attributes to store contexts
    def __init__(self, key_name, invalid_value, message="Invalid configuration value"):
        self.key_name = key_name
        self.invalid_value = invalid_value
        full_message = f"{message} for key '{key_name}': received '{invalid_value}'"
        super().__init__(full_message)


try:
    raise ConfigValueError("timeout", -5, message="Timeout cannot be negative")
except ConfigValueError as e:
    print(f"{e}")
    print(f"\t-> key: {e.key_name}")
    print(f"\t-> value: {e.invalid_value}")

Timeout cannot be negative for key 'timeout': received '-5'
	-> key: timeout
	-> value: -5


## Raising and Catching Enhanced Custom Exceptions
- Raise custom exceptions by instantiating them with relevant arguments: `raise MyError(arg1, arg2)`.  
- In `except` blocks, catch specific exceptions and access their attributes for tailored recovery or logging.  
- Fallback `except BaseError:` catches any related subclass if no more specific handler exists.  

In [ ]:
class DeploymentError(Exception):
    """Base class for deployment-related errors."""
    pass


class InvalidEnvironmentError(DeploymentError):
    """Raised when environment is invalid."""
    def __init__(self, env_name, allowed_envs):
        self.env_name = env_name
        self.allowed_envs = allowed_envs
        super().__init__(f"Invalid environment '{env_name}'. Allowed values: {allowed_envs}")


class PackageMissingError(DeploymentError):
    """Raised when required packages are missing."""
    def __init__(self, package_name, host):
        self.package_name = package_name
        self.host = host
        super().__init__(f"Package '{package_name}' is missing on host {host}.")


def deploy_app(environment, package):
    allowed_envs = ["staging", "production"]

    # We can pass the context values to the custom exception
    if environment not in allowed_envs:
        raise InvalidEnvironmentError(environment, allowed_envs)

    # We can pass the context values to the custom exception
    if environment == "production" and package == "critical-lib":
        raise PackageMissingError(package, f"server-{environment}")

    print(f"Deployment to {environment} with package {package} succeeded.")


for env, pkg in [("dev", "tool"), ("production", "critical-lib"), ("staging", "tool")]:
    try:
        deploy_app(env, pkg)
    except DeploymentError as e:
        print(e)


came here
Invalid environment 'dev'. Allowed values: ['staging', 'production']
came here
Package 'critical-lib' is missing on host server-production.
Deployment to staging with package tool succeeded.


## Hands-on Exercise

In [18]:
def validate_config(config_data):
    if not isinstance(config_data, dict):
        raise TypeError("Configuration must be a dictionary.")
 
    mem_limit = config_data.get("memory_limit_mb")
    if mem_limit is None:
        raise ValueError("Mandatory key 'memory_limit_mb' is missing.")
 
    if not (isinstance(mem_limit, int) and 256 <= mem_limit <= 4096):
        raise ValueError(f"Memory limit {mem_limit} is outside the allowed range (256-4096).")
 
    print(f"Config valid: {mem_limit}MB")
 
 
configs = [
    {"memory_limit_mb": 1024},
    {"memory_limit_mb": 128},
    {"cpu_cores": 4},
    "memory_limit_mb: 512"
]
 
for config in configs:
    try:
        validate_config(config)
    except (ValueError, TypeError) as e:
        print(f"Error: {e}")

Config valid: 1024MB
Error: Memory limit 128 is outside the allowed range (256-4096).
Error: Mandatory key 'memory_limit_mb' is missing.
Error: Configuration must be a dictionary.


In [6]:
class ProvisionError(Exception):
    """Base class for provisioning failures."""
    pass


class DiskSpaceError(ProvisionError):
    """Raised when there is not enough disk space."""
    def __init__(self, required, available):
        super().__init__(f"Not enough disk. Required: {required}GB, Available: {available}GB")


class PermissionsError(ProvisionError):
    """Raised due to file system permission issues."""
    pass


def provision_file(size_gb, path):
    if size_gb > 100:
        raise DiskSpaceError(required=size_gb, available=100)
    if "/root/" in path:
        raise PermissionsError(f"Cannot write to protected path: {path}")
    print("Provisioning successful.")


try:
    provision_file(size_gb=50, path="/root/data.bin")
except DiskSpaceError as e:
    print(f"Caught Disk Error: {e}")
except ProvisionError as e:
    print(f"Caught Provision Error: {e}")
except Exception:
    print("Caught a generic exception.")

Caught Provision Error: Cannot write to protected path: /root/data.bin


In [11]:
class ConfigSyntaxError(ValueError):
    def __init__(self, message, line_num, text):
        self.line = line_num
        self.text = text
        full_msg = f"Syntax error on line {line_num}: {message}"
        super().__init__(full_msg)
 
def parse_config(lines):
    for i, line in enumerate(lines, 1):
        if "=" not in line:
            raise ConfigSyntaxError("Missing '=' assignment", i, line)
    return "Parsed OK"
 
config_text = ["host=server.local", "port", "timeout=30"]
 
try:
    parse_config(config_text)
except ConfigSyntaxError as e:
    print(e)
    print(f"-> Problematic text: '{e.text}'")


Syntax error on line 2: Missing '=' assignment
-> Problematic text: 'port'
